In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("..")

In [2]:
from pathlib import Path

from librep.datasets.kuhar.raw import RawKuHar
from librep.datasets.kuhar.views import KuHarDatasetGenerator, KuHarRawIterator

In [3]:
dataset_dir = Path("../data/datasets/KuHar/1.Raw_time_domain_data")
kuhar_dataset = RawKuHar(dataset_dir, download=False)
kuhar_dataset

KuHar Dataset at: '../data/datasets/KuHar/1.Raw_time_domain_data'

In [4]:
iterator = KuHarRawIterator(kuhar_dataset)
iterator

KuharView: users=89, activities=18

In [5]:
kuhar_v1 = KuHarDatasetGenerator(iterator, time_window=300, window_overlap=0)
kuhar_v1

Dataset generator: time_window=300, overlap=0, labels=class

In [6]:
train, validation, test = kuhar_v1.create_datasets(
    train_size=0.7,
    validation_size=0.1,
    test_size=0.2,
    ensure_distinct_users_per_dataset=False,
    balance_samples=False,
)
print(train)
print(validation)
print(test)

Generating full df over KuHar View: 1945it [01:26, 22.49it/s]


PandasDataset: samples=14279, features=1800, label_column='class'
PandasDataset: samples=1424, features=1800, label_column='class'
PandasDataset: samples=4334, features=1800, label_column='class'


In [7]:
from librep.utils.dataset import load_full_data

x, y = load_full_data(train, return_X_y=True)

In [8]:
x.shape, y.shape

((14279, 1800), (14279,))

In [11]:
x[10], y[10]

(array([6.3103285 , 9.832279  , 7.780205  , ..., 0.5237436 , 0.2836736 ,
        0.08453153]),
 12)